## Integrate data from Notebooks with Azure Data Factory or Azure Synapse Pipelines
- Local & Global Parameter
- Parameter Driven

In [ ]:
%%pyspark
df = (
         spark.read.load('abfss://root@adlesilabs.dfs.core.windows.net/sale-poc/sale-20170501.csv', format='csv'
        ,header=True
        ,inferschema=True
)
)
display(df.limit(3))

## Goal is to return top 5 best customers

In [ ]:
SalePOC = df.select("CustomerID","Quantity","TotalAmount","ProfitAmount")

In [ ]:
display(SalePOC.limit(3))

A SQL Guy

In [ ]:
SalePOC.createOrReplaceTempView("Top5BestCustomers")

In [ ]:
%%sql
SELECT CustomerID
      ,SUM(Quantity) AS SubTotalQuantity
      ,CAST(SUM(TotalAmount) AS DECIMAL(18,2)) AS SubTotalAmount
      ,CAST(SUM(ProfitAmount) AS DECIMAL(18,2)) AS SubProfitAmount
FROM Top5BestCustomers
GROUP BY CustomerID
ORDER BY SubProfitAmount DESC
LIMIT 5

In [ ]:
spark.sql("""
SELECT CustomerID
      ,SUM(Quantity) AS SubTotalQuantity
      ,CAST(SUM(TotalAmount) AS DECIMAL(18,2)) AS SubTotalAmount
      ,CAST(SUM(ProfitAmount) AS DECIMAL(18,2)) AS SubProfitAmount
FROM Top5BestCustomers
GROUP BY CustomerID
ORDER BY SubProfitAmount DESC
LIMIT 5
""").show()

In [ ]:
Top5BestCustomer = spark.sql("""
SELECT CustomerID
      ,SUM(Quantity) AS SubTotalQuantity
      ,CAST(SUM(TotalAmount) AS DECIMAL(18,2)) AS SubTotalAmount
      ,CAST(SUM(ProfitAmount) AS DECIMAL(18,2)) AS SubProfitAmount
FROM Top5BestCustomers
GROUP BY CustomerID
ORDER BY SubProfitAmount DESC
LIMIT 5
""")

In [ ]:
display(Top5BestCustomer)

In [ ]:
dateFormat = "%Y%m%d%H%M"
import datetime
ts=spark.sql(""" select current_timestamp() as ctime """).collect()[0]["ctime"]


In [ ]:
ld_datetime = ts.strftime(dateFormat)

In [ ]:
Top5BestCustomer.write.parquet('abfss://root@adlesilabs.dfs.core.windows.net/IntegrateNotebooks/SalesOrderHeader/' + str(ld_datetime))

# Clean Up

In [ ]:
 #mssparkutils.fs.rm('abfss://root@adlesilabs.dfs.core.windows.net/IntegrateNotebooks/SalesOrderHeader', True) 